## Chassis Example Notebooks
Welcome to the examples section for [Chassis](https://chassis.ml), which contains notebooks that auto-containerize models built using the most common machine learning (ML) frameworks. 

#### What is Chassis?
Chassis allows you to automatically create a Docker container from your model code and push that container image to a Docker registry. All you need is your model loaded into memory and a few lines of Chassis code! Our example bank is here to provide reference examples for many common ML frameworks.  

Can't find the framework you are looking for or need help? Fork this repository and open a PR, or list the desired framework in a new issue. We're always interested in growing this example bank! 

The primary maintainers of Chassis also actively monitor our [Discord Server](https://discord.gg/cHpzY9yCcM), so feel free to join and ask any questions you might have. We'll be there to respond and help out promptly.

In [31]:
import time
import chassisml
from io import StringIO
import numpy as np
import pandas as pd
import getpass
import json
from xgboost import XGBClassifier
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

## Enter credentials
Dockerhub creds and Modzy API Key

In [32]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')

docker hub username········
docker hub password········


## Load sample data and Train XGBoost Model

In [33]:
# load data
data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data['data'], data['target'], test_size=.2)
# create model instance
bst = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
# fit model
bst.fit(X_train, y_train)
# make predictions
preds = bst.predict(X_test)

In [34]:
classes = data['target_names'].tolist()

## Write process function

* Must take bytes as input
* Preprocess bytes, run inference, postprocess model output, return results

In [35]:
def process(input_bytes):
    # load data
    inputs = pd.read_csv(StringIO(str(input_bytes, "utf-8")))    
    
    # run inference
    preds = bst.predict_proba(inputs)
    
    # structure results
    inference_result = {
        "classPredictions": [
            {"row": i+1, "class": classes[np.argmax(pred)], "score": np.max(pred)} for i, pred in enumerate(preds)
        ]
    }

    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [36]:
chassis_client = chassisml.ChassisClient("https://chassis.app.modzy.com")

## Create and test Chassis model
* Requires `process_fn` defined above

In [37]:
# create Chassis model
chassis_model = chassis_client.create_model(process_fn=process)

# test Chassis model locally (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './data/iris_data.csv'
results = chassis_model.test(sample_filepath)
print(results)

b'{"data":{"result":{"classPredictions":[{"row":1,"class":"virginica","score":0.9089527726173401},{"row":2,"class":"virginica","score":0.9089527726173401},{"row":3,"class":"versicolor","score":0.9293136596679688},{"row":4,"class":"versicolor","score":0.9293136596679688}]},"explanation":null,"drift":null}}'


In [38]:
# manually construct conda environment to pass to Chassis job

# NOTE: if you define your environment manually like this, the "chassisml" package must be included within the pip depedencies
env = {
    "name": "sklearn-chassis",
    "channels": ['conda-forge'],
    "dependencies": [
        "python=3.8.5",
        {
            "pip": [
                "scikit-learn",
                "numpy",
                "chassisml",
                "xgboost",
                "pandas"
            ] 
        }
    ]
}

## Publish model to Dockerhub
Need to provide model name, model version, Dockerhub credentials

In [41]:
MODEL_NAME = "XGBoost Iris Classification"
start_time = time.time()
response = chassis_model.publish(
    model_name=MODEL_NAME,
    model_version="0.0.1",
    registry_user=dockerhub_user,
    registry_pass=dockerhub_pass,
    conda_env=env
)

Starting build job... Ok!


In [42]:
job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)
end_time = time.time()
if final_status['status']['succeeded'] == 1:
    print("Job Completed in {} minutes.\n\nView your new container image here: https://hub.docker.com/repository/docker/{}/{}".format(round((end_time-start_time)/60, 5), dockerhub_user, "-".join(MODEL_NAME.lower().split(" "))))
else:
    print("Job Failed. See logs below:\n\n{}".format(final_status['logs']))

Job Completed in 5.38254 minutes.

View your new container image here: https://hub.docker.com/repository/docker/bmunday131/xgboost-iris-classification
